|        |        |        |
|--------|--------|--------|
![H-BRS](logos/h-brs.png) | ![A2S](logos/a2s.png) | ![b-it](logos/b-it.png) |

# Autonomous Mobile Robots

# AMR Assignment 2

### General information

* Please do not add or delete any cells. Answers belong into the already provided cells (below the question).
* If a function is given (either as a signature or a full function), you should not change the name, arguments, or return value of the function.
* If you encounter empty cells underneath the answer that can not be edited, please ignore them; they are for testing purposes.
* Please note that variables declared in the notebook cells have global scope. To make sure your assignment works correctly as a whole, please restart the kernel and run all cells before submitting (e.g. via *Kernel -> Restart & Run All*).
* Code cells where you are supposed to give your answer often include the line  ```raise NotImplementedError```. This makes it easier to automatically grade answers. Once you fill out a function, please delete this line.

### Submission

Please make sure to write all your team members 2s IDs in the cell below before submission. Please submit your notebook via the JupyterHub web interface (in the main view -> Assignments -> Submit). If it is a group assignment, please make only one submission per group (for easier bookkeeping, it is best if this is always the same team member).

### Questions about the assignment

If you have questions about the assignment, you are encouraged to post them in the LEA forum. Proactive discussions lead to better understanding. Let's keep the forum active.

## Team members (2s IDs):

YOUR ANSWER HERE

*mmemon2s
*
*

# A. Multi-sensor Data Visualisation, Analysis, and Filtering [50 Points]

In this assignment, you will visualise the data from different sensors on the Robile; this includes 2D and 3D LiDARs, an IMU, and an RGB-D camera. You will particularly get accustomed to visualisation tools other than the standard ROS visualsation tool Rviz (in particular [`Foxglove`](https://foxglove.dev) and [`PlotJuggler`](https://github.com/facontidavide/PlotJuggler)), which you will use to detect some anomalies added to the LiDAR and RGB-D sensors, namely you will need to detect the approximate time or time range when these anomalies are introduced.

In addition, you will apply different filtering methods to the data, such as mean  and median filtering over the 2D LiDAR data and IMU data.

To work on this assignment, you need to download a ROS2 bag file that includes some data collected from the robot. The URL and password are as follows:
* Bag file (3.1GB): https://nx21454.your-storageshare.de/s/jfTkS9NLtG6gWGp
* Share password: `r0b!l3Rosbag?Da(ta*`

**Note:** As the ROS-related packages are not installed in the e2x Jupyter server, please **download this notebook and work on it locally**. Then, for submitting, update the code cells in this notebook (on the e2x server) and include appropriate plots with your submission. **Please do NOT upload the bag file with the submission**.

Let us first import all necessary packages. You can uncomment the desired packages and import any other required packages.

In [ ]:
## Uncomment desired packages

# import rclpy
# import rosbag2_py
# from rosbag2_py import StorageOptions, ConverterOptions, SequentialReader
# import rosbags
# from rosbags.serde import deserialize_cdr
# from rosbags.rosbag2 import Reader
# from rclpy.serialization import deserialize_message

# from sensor_msgs.msg import LaserScan
# from sensor_msgs.msg import PointCloud2
# from sensor_msgs.msg import Image
# from tf2_msgs.msg import TFMessage

# import tf2_ros
# import tf_transformations
# from geometry_msgs.msg import Quaternion

# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

## LiDAR visualisation [20 Points]

An anomaly is introduced by covering the LiDAR sensor by an opaque object, because of which we lose the laser scan data for approximately 2 seconds. Using the `Foxglove` visualisation tool, find the approximate time period in which the 2D LiDAR was covered. After visualising the data, please take a screenshot of it and upload the screenshot to the `img` directory. Then, embed the screenshot in the markdown cell below using:

```
![lidar_visualisation_screenshot](img/file_name.png)
```

where `file_name` is the image file name, and comment on the results.

**Notes:**
* In `Foxglove` you can directly upload the `.db3` rosbag file and use the different panel options to visualise the data.
* Due to the size of the data, it might not be possible to visualise all the data together in `Foxglove` based on your system configuration. Please thus only visualise the necessary data for the tasks. It is also recommended to download the tool locally instead of using the web application for better handling of large amounts of data.
* In `PlotJuggler`, you can upload the `.yaml` file located in the directory of the `.db3` bag file and select the topics to be visualised in the prompt that is displayed after selecting yaml file. The corresponding fields can then be dragged and dropped for visualisation.

YOUR ANSWER HERE

## Visualisation of 3D LiDAR data from a RoboSense LiDAR [10 Points]

Using `Foxglove`, take a screenshot of the 3D LiDAR data from the RoboSense at some arbitrary time instance and embed the image in the below cell.

YOUR ANSWER HERE

## RGB image visualisation [10 Points]

Take a screenshot of the visualisation tool where an image from the RealSense camera is visible and embed the image in the cell below.

**Hint:** You might take a look at different options under panel tab in `Foxglove`.

YOUR ANSWER HERE

## IMU data visualisation [10 Points]

To record the scenario of impact on the robot, it is "kicked" after it comes to a halt. Using the IMU readings, find the time instance at which the robot detects this impact. As before, find the approximate time period in which the impact was detected. After visualising the data, please take a screenshot of it and embed it in the cell below.

You can use either `Foxglove` or `PlotJuggler` to plot the IMU acceleraion readings. If you use `Foxglove`, it is helpful to simultaneously look at the robot moving and the corresponding IMU readings. A sample visualisation is shown below:

![imu-visualisation-sample](img/imu-visualisation-sample.png)

**Note**: Spikes in the IMU data are also caused by start of motion of the robot, so you should label the spikes in your screenshot accordingly.

YOUR ANSWER HERE

# B. Filtering [50 Points]

As discussed in our lectures, sensor readings generally contain outliers and measurement noise. In this exercise, you will implement two data filtering techniques, namely mean and median filters, on the 2D LiDAR data and the IMU data.

As in the previous part, please work on your local machine, and then only copy your code and include any necessary screenshots in this notebook.

### Extraction of data from bag files [10 Points]

Before looking into these filters, we will extract the necessary data from the bag files. In the cell below, extract both 2D LiDAR data and IMU acceleration data from the bag file. In particular, read the 2D LiDAR data for a particular time instance closest to some user-specified time, and also read the full IMU data from the bag file.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Median filter of 2D LiDAR data [10 Points]

The idea behind a median filter is to use a sliding window of specific length in order to replace the central point in the window by the median of the points in the whole window. The median filter is a low-pass filter that removes outliers in the sensor data.

Your task is to apply the median filter on the 2D LiDAR data at a predefined time instance (you can experiment with the size of the sliding window). For this particular time instance, plot the `x` vs `y` coordinates of the 2D LiDAR data before and after applying the filter.

Please refer to this [wiki page](https://en.wikipedia.org/wiki/Median_filter) for an example of filtering 1D data.

**Note:** Please remember to include a title, grid, axes labels, and a legend in your plot.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Median filtering of IMU data [10 Points]

Now, perform median filtering of the IMU acceleration over the y-axis. Then, plot the time vs. acceleration before and after filtering.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Mean filtering of 2D LiDAR data [10 Points]

The mean filter operates similarly to the median filter; however, the point under consideration is replaced by the mean of all points in the same window. This is generally used to smoothen the noise in the data rather than eliminate it fully.

Your task is to apply the mean filter on the 2D LiDAR data at the same time instance as before (again, you can experiment with the size of the sliding window). For this particular time instance, plot the `x` vs `y` coordinates of the 2D LiDAR data before and after applying the filter.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()


### Mean filtering of 2D IMU data [10 Points]

Finally, perform mean filtering of the IMU acceleration over the y-axis. Then, plot the time vs. acceleration before and after filtering.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()